# Neural Networks

This neural network is an attempt to create a model without using scikit-learn. This was intended to lower the slug size of my heroku upload, but it still wasn't small enough. The resulting model is much larger than the earlier model with only dense layers. The network also does not perform as well as the earlier model. I would like to come back to this later and perform a wider gridsearch and different layers, as I believe an RNN such as this has the potential to ouytperform the more basic model I created earlier.

In [1]:
import pandas as pd

In [2]:
y_train = pd.read_feather('../data/processed/y_train.feather')['voted_up'].to_numpy()
X_train = pd.read_feather('../data/processed/X_train_preprocessed.feather').to_numpy()

In [3]:
y_test = pd.read_feather('../data/processed/y_test.feather')['voted_up'].to_numpy()
X_test = pd.read_feather('../data/processed/X_test_preprocessed.feather').to_numpy()

## Baseline Neural Network

In [4]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
from tensorflow.keras.layers import Dense, Dropout, Embedding, GlobalMaxPool1D, LSTM
from tensorflow.keras import Sequential

In [5]:
from keras.preprocessing import text, sequence

In [6]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train.flatten().tolist())
tokenized_list_train = tokenizer.texts_to_sequences(X_train.flatten().tolist())
tokenized_list_test = tokenizer.texts_to_sequences(X_test.flatten().tolist())
X_train_tokenized = sequence.pad_sequences(tokenized_list_train, maxlen=256)
X_test_tokenized = sequence.pad_sequences(tokenized_list_test, maxlen=256)

In [7]:
model = Sequential()

# hidden layers
model.add(Embedding(20000, 512))
model.add(LSTM(50, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))

# output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
model.fit(X_train_tokenized, y_train, epochs=3, batch_size=64, validation_data=(X_test_tokenized, y_test))
model.evaluate(X_test_tokenized, y_test)

Epoch 1/3
914/914 [==============================] - 489s 532ms/step - loss: 0.4164 - accuracy: 0.8320 - val_loss: 0.2504 - val_accuracy: 0.8979
Epoch 2/3
914/914 [==============================] - 484s 529ms/step - loss: 0.2091 - accuracy: 0.9180 - val_loss: 0.2354 - val_accuracy: 0.9023
Epoch 3/3
457/457 [==============================] - 33s 73ms/step - loss: 0.2578 - accuracy: 0.9014


[0.25776994228363037, 0.9013679623603821]

0.9013679623603821

## Hyperparameter Tuning

In [9]:
from talos import Scan

In [16]:
def dense_network(x_train, y_train, x_val, y_val, params):
    model = Sequential()

    # hidden layers
    model.add(Embedding(20000, 512))
    model.add(LSTM(params['lstm'], return_sequences=True))
    model.add(GlobalMaxPool1D())
    model.add(Dropout(params['dropout']))
    model.add(Dense(params['dense'], activation='relu'))
    model.add(Dropout(params['dropout']))
    # output layer
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    out = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=5,
                    verbose=1)

    return out, model

In [11]:
params = {'dropout': [0.25, 0.5, 0.75], 
          'dense': [10, 50, 100], 
          'lstm': [10, 50, 100]}

In [17]:
results = Scan(X_train_tokenized, y_train, params=params, model=dense_network, experiment_name='grid')
results.best_model(metric='accuracy')



  0%|          | 0/27 [00:00<?, ?it/s]

Epoch 1/5
1280/1280 [==============================] - 356s 277ms/step - loss: 0.4126 - accuracy: 0.8289 - val_loss: 0.2686 - val_accuracy: 0.8888
Epoch 2/5
1280/1280 [==============================] - 352s 275ms/step - loss: 0.2342 - accuracy: 0.9095 - val_loss: 0.2596 - val_accuracy: 0.8949
Epoch 3/5
1280/1280 [==============================] - 349s 273ms/step - loss: 0.1673 - accuracy: 0.9392 - val_loss: 0.2783 - val_accuracy: 0.8952
Epoch 4/5
1280/1280 [==============================] - 346s 271ms/step - loss: 0.1279 - accuracy: 0.9565 - val_loss: 0.3235 - val_accuracy: 0.8935
Epoch 5/5
1280/1280 [==============================] - 346s 270ms/step - loss: 0.0998 - accuracy: 0.9678 - val_loss: 0.3490 - val_accuracy: 0.8922


  0%|          | 0/27 [42:26<?, ?it/s]


  4%|▎         | 1/27 [29:10<12:38:24, 1750.18s/it]

Epoch 1/5
1280/1280 [==============================] - 494s 385ms/step - loss: 0.4615 - accuracy: 0.8170 - val_loss: 0.2636 - val_accuracy: 0.8910
Epoch 2/5
1280/1280 [==============================] - 488s 381ms/step - loss: 0.2457 - accuracy: 0.9127 - val_loss: 0.2530 - val_accuracy: 0.8959
Epoch 3/5
1280/1280 [==============================] - 493s 385ms/step - loss: 0.1694 - accuracy: 0.9452 - val_loss: 0.2623 - val_accuracy: 0.8935
Epoch 4/5
1280/1280 [==============================] - 491s 384ms/step - loss: 0.1140 - accuracy: 0.9629 - val_loss: 0.3127 - val_accuracy: 0.8927
Epoch 5/5
1280/1280 [==============================] - 489s 382ms/step - loss: 0.0834 - accuracy: 0.9755 - val_loss: 0.3921 - val_accuracy: 0.8925




  7%|▋         | 2/27 [1:10:06<15:02:18, 2165.54s/it]

...



 93%|█████████▎| 25/27 [18:09:10<1:24:45, 2542.55s/it]

Epoch 1/5
1280/1280 [==============================] - 498s 387ms/step - loss: 0.4626 - accuracy: 0.8096 - val_loss: 0.2850 - val_accuracy: 0.8816
Epoch 2/5
1280/1280 [==============================] - 493s 385ms/step - loss: 0.2556 - accuracy: 0.9025 - val_loss: 0.2630 - val_accuracy: 0.8843
Epoch 3/5
1280/1280 [==============================] - 491s 384ms/step - loss: 0.1998 - accuracy: 0.9250 - val_loss: 0.2683 - val_accuracy: 0.8885
Epoch 4/5
1280/1280 [==============================] - 492s 385ms/step - loss: 0.1630 - accuracy: 0.9415 - val_loss: 0.2999 - val_accuracy: 0.8849
Epoch 5/5
1280/1280 [==============================] - 492s 385ms/step - loss: 0.1267 - accuracy: 0.9572 - val_loss: 0.3071 - val_accuracy: 0.8840




 96%|█████████▋| 26/27 [18:50:17<41:59, 2519.94s/it]  

Epoch 1/5
1280/1280 [==============================] - 747s 582ms/step - loss: 0.4578 - accuracy: 0.8082 - val_loss: 0.2887 - val_accuracy: 0.8761
Epoch 2/5
1280/1280 [==============================] - 750s 586ms/step - loss: 0.2652 - accuracy: 0.8943 - val_loss: 0.2746 - val_accuracy: 0.8811
Epoch 3/5
1280/1280 [==============================] - 751s 587ms/step - loss: 0.2261 - accuracy: 0.9130 - val_loss: 0.2907 - val_accuracy: 0.8813
Epoch 4/5
1280/1280 [==============================] - 752s 588ms/step - loss: 0.1655 - accuracy: 0.9412 - val_loss: 0.2904 - val_accuracy: 0.8885
Epoch 5/5
1280/1280 [==============================] - 751s 587ms/step - loss: 0.1154 - accuracy: 0.9589 - val_loss: 0.3657 - val_accuracy: 0.8895




100%|██████████| 27/27 [19:52:50<00:00, 2650.75s/it]


In [19]:
df = pd.read_csv('grid/030421195726.csv')
df.sort_values('val_accuracy', ascending=False)

,round_epochs,loss,accuracy,val_loss,val_accuracy,dense,dropout,lstm
10,5,0.062046,0.978330,0.452788,0.894887,50,0.25,50
19,5,0.055093,0.980822,0.474116,0.893747,100,0.25,50
2,5,0.068779,0.978257,0.401478,0.893405,10,0.25,100
9,5,0.096420,0.966506,0.330394,0.893120,50,0.25,10
23,5,0.071000,0.974861,0.382407,0.892892,100,0.50,100
1,5,0.087180,0.973615,0.392137,0.892493,10,0.25,50
0,5,0.105851,0.964625,0.348987,0.892208,10,0.25,10
14,5,0.074258,0.973689,0.395824,0.891923,50,0.50,100
20,5,0.052169,0.981555,0.436970,0.891752,100,0.25,100
11,5,0.056222,0.979918,0.464181,0.891410,50,0.25,100


In [22]:
df.groupby(['lstm']).mean().sort_values('val_accuracy', ascending=False)

,round_epochs,loss,accuracy,val_loss,val_accuracy,dense,dropout
lstm,,,,,,,
100,5.0,0.108989,0.956555,0.382181,0.890612,53.333333,0.5
50,5.0,0.119361,0.957331,0.387984,0.887191,53.333333,0.5
10,5.0,0.186335,0.927347,0.314521,0.884664,53.333333,0.5


In [21]:
df.groupby(['dense']).mean().sort_values('val_accuracy', ascending=False)

,round_epochs,loss,accuracy,val_loss,val_accuracy,dropout,lstm
dense,,,,,,,
50,5.0,0.123223,0.954215,0.372608,0.888528,0.5,53.333333
10,5.0,0.176798,0.930398,0.341215,0.887483,0.5,53.333333
100,5.0,0.114665,0.956620,0.370862,0.886457,0.5,53.333333


In [23]:
df.groupby(['dropout']).mean().sort_values('val_accuracy', ascending=False)

,round_epochs,loss,accuracy,val_loss,val_accuracy,dense,lstm
dropout,,,,,,,
0.25,5.0,0.074652,0.974677,0.405665,0.891923,53.333333,53.333333
0.50,5.0,0.116911,0.958480,0.359308,0.886628,53.333333,53.333333
0.75,5.0,0.223122,0.908077,0.319712,0.883917,53.333333,53.333333


## Final Model

In [7]:
model = Sequential()

# hidden layers
model.add(Embedding(20000, 512))
model.add(LSTM(50, return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.25))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.25))

# output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
model.fit(X_train_tokenized, y_train, epochs=10, batch_size=32, validation_split=0.1)

Epoch 1/10
1645/1645 [==============================] - 582s 353ms/step - loss: 0.3683 - accuracy: 0.8483 - val_loss: 0.2536 - val_accuracy: 0.8940
Epoch 2/10
1645/1645 [==============================] - 569s 346ms/step - loss: 0.1798 - accuracy: 0.9289 - val_loss: 0.2503 - val_accuracy: 0.8969
Epoch 3/10
1645/1645 [==============================] - 567s 345ms/step - loss: 0.1151 - accuracy: 0.9581 - val_loss: 0.3334 - val_accuracy: 0.8878
Epoch 4/10
1645/1645 [==============================] - 559s 340ms/step - loss: 0.0733 - accuracy: 0.9731 - val_loss: 0.3499 - val_accuracy: 0.8921
Epoch 5/10
1645/1645 [==============================] - 577s 351ms/step - loss: 0.0500 - accuracy: 0.9829 - val_loss: 0.4385 - val_accuracy: 0.8895
Epoch 6/10
1645/1645 [==============================] - 577s 351ms/step - loss: 0.0361 - accuracy: 0.9874 - val_loss: 0.4749 - val_accuracy: 0.8846
Epoch 7/10
1645/1645 [==============================] - 564s 343ms/step - loss: 0.0324 - accuracy: 0.9888 - val_

In [9]:
model.evaluate(X_test_tokenized, y_test)

457/457 [==============================] - 31s 67ms/step - loss: 0.6396 - accuracy: 0.8946


[0.639572024345398, 0.8945964574813843]

In [10]:
model.save('../final_model/model')
model.save_weights('../final_model/_weights')

INFO:tensorflow:Assets written to: ../final_model/model\assets


INFO:tensorflow:Assets written to: ../final_model/model\assets
